In [ ]:
#@title Install babashka https://github.com/babashka/babashka
!(curl -s https://raw.githubusercontent.com/babashka/babashka/master/install) >> .install-babashka
!. .install-babashka

In [ ]:
#@title Babashka CLI call
!ls | bb -i '(filter fs/directory? *input*)'

In [ ]:
#@title run Babashka clojure REPL
!bb

In [ ]:
#@title run scripts/bb.clj
!curl https://raw.githubusercontent.com/roterski/colab-clojure-babashka/main/scripts/bb.clj | bb

In [ ]:
#@title Run clojure code from input
clj_code = "(do (def a 5) (+ a 4) )" #@param {type:"string"}

!!bb -i "$clj_code"

# Running clojure snippets


Manual steps:
- [ ] Click Copy to Drive button in the top bar to create this notebook copy in your drive
- [ ] Switch to newly created tab and re-run install babashka step there


### Clojure snippet 1
clj-1:
```clojure
#!/usr/bin/env bb

(def now (java.time.ZonedDateTime/now))
(def LA-timezone (java.time.ZoneId/of "America/Los_Angeles"))
(def LA-time (.withZoneSameInstant now LA-timezone))
(def pattern (java.time.format.DateTimeFormatter/ofPattern "HH:mm"))
(println (.format LA-time pattern))
```
clj-2:
```clojure
(println (+ 3 5))
```

In [2]:
#@title authenticate to Colab and define run_clj function
from google.colab import auth
auth.authenticate_user()

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from oauth2client.client import GoogleCredentials
import json

gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
#@markdown Take the Colab Nodebook url from URL bar of this browser tab:
colab_notebook_url = '' #@param {'type': 'string'}

file_id = colab_notebook_url.split("https://colab.research.google.com/drive/")[-1].split("#")[0]


def run_clj(snippet_id):
  file = drive.CreateFile({'id': file_id})
  downloaded = file.GetContentString()
  print("Running clojure code:\n")

  file = open("bb.clj", "w+")
  clojure_code = False
  for cell in json.loads(downloaded)['cells']:
    if cell['cell_type'] == 'markdown':
      is_chosen_snippet = False
      for line in cell['source']:
        if line == f'clj-{snippet_id}:\n':
          is_chosen_snippet = True
        if clojure_code == True and line == '```\n':
          clojure_code = False
          is_chosen_snippet = False
        if clojure_code and is_chosen_snippet:
          print(line)
          file.write(line)
        if line == f'```clojure\n':
          clojure_code = True
        
  file.close()
  print("=>")
  !bb bb.clj


In [ ]:
snippet_id = 2 #@param {'type': 'integer'}
run_clj(snippet_id)

# Connecting remote ssh nREPL

1. use to https://github.com/WassimBenzarti/colab-ssh to create a ssh tunnel to colab env
2. it requires creating ngrok (or cloudflared account) and obtaining a token
3. once ssh tunnel is open, you need to configure your local editor (VS Code) Remote SSH and connect to it
4. install Calva there, run `bb --nrepl-server`
5. connect Calva REPL to running babashka nREPL

In [ ]:
!pip install colab_ssh --upgrade

In [ ]:
from colab_ssh import launch_ssh

ngrok_token = "" #@param {'type': 'string'}
password = "" #@param {'type': 'string'}
launch_ssh(token=ngrok_token, password=password)